In [ ]:
import folium
import pandas as pd

In [ ]:
seoul_map = folium.Map(location=[37.55,126.98], zoom_start=12)
seoul_map

In [ ]:
seoul_map2 = folium.Map(location=[37.55,126.98],  zoom_start=12, tiles='Cartodb Positron')
seoul_map2

In [ ]:
seoul_map3 = folium.Map(location=[37.55,126.98], tiles="Cartodb dark_matter", zoom_start=12)
seoul_map3

## 서울대학교 마커 찍기

In [ ]:
import pandas as pd

university_data = {
    'University': ['Seoul National University', 'korea University'],
    'lat': [37.4590, 37.59080],
    'lng': [126.9519, 127.0278],
}

df = pd.DataFrame(university_data)


In [ ]:
folium.Marker([df.lat[0], df.lng[0]], popup=df.University[0]).add_to(seoul_map2)
seoul_map2

In [ ]:
folium.CircleMarker([df.lat[1], df.lng[1]], popup=df.University[1]).add_to(seoul_map2)
seoul_map2

# 과제 스타벅스에서 주솟값 크롤링 후 지도에 마커 표시

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

url = 'https://www.starbucks.co.kr/store/store_map.do'
res = requests.get(url)
soup = bs(res.content, 'html.parser')

result = soup.find('#mCSB_3_container > ul > li:nth-child(451) > p')
data = []
print(result)
for info in result:
  data.append(info.text)

data

None


TypeError: ignored

In [ ]:
import requests
from pandas.io.json import json_normalize

# 스타벅스 사이트에서 위치정보를 수집
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=5XLKAT2F0K'
request = requests.post(targetSite, data={
    'ins_lat': 37.5108295,           # 위도
    'ins_lng': 127.02928809999999,   # 경도
    'p_sido_cd': '01',               # 시,도 분류 코드
    'p_gugun_cd': '',                # 시,군,구 분류 코드
    'in_biz_cd': '',                 # 알 수 없는 정보(불필요)
    'iend': 2000,                    # 불러올 최대 매장 수
    'set_date': ''                   # 알 수 없는 정보(불필요)
})

star_json = request.json()
star_df = json_normalize(star_json, 'list')


star_df.to_csv('starbucks_locations.csv', index=False)


In [ ]:
star_df = pd.read_csv('/content/starbucks_locations.csv', encoding='utf-8')


star_data = {
    'name': [],
    'lat': [],
    'lot': [],
}

for index, row in star_df.iterrows():
    star_data['name'].append(row['s_name'])
    star_data['lat'].append(row['lat'])
    star_data['lot'].append(row['lot'])

star_data


In [ ]:
for i in range(len(star_data['name'])):
  folium.Marker([star_data['lat'][i], star_data['lot'][i]], popup=star_data['name'][i]).add_to(seoul_map)

seoul_map

In [17]:
from geopy.geocoders import Nominatim
import pandas as pd

data_path = '/content/서울특별시 자치구별 코로나19 백신접종현황.csv'
data = pd.read_csv(data_path, encoding='cp949')

df = pd.DataFrame(data)

districts = df['자치구']

latitudes = []
longitudes = []

geolocator = Nominatim(user_agent="my_geocoder")

for district in districts:
    location = geolocator.geocode(f"{district}, 서울")
    if location:
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)
    else:
        latitudes.append(None)
        longitudes.append(None)

df['lat'] = latitudes
df['lot'] = longitudes

start_col = '2-6월 1차접종자'
end_col = '10월말(누적) 추가접종'

df['total'] = df.loc[:, start_col:end_col].sum(axis=1)

In [20]:
df.head()

,자치구,2-6월 1차접종자,2-6월 2차접종자,7월말(누적) 1차접종자,7월말(누적) 2차접종자,8월말(누적) 1차접종자,8월말(누적) 2차접종자,9월말(누적) 1차접종자,9월말(누적) 2차접종자,10월말(누적) 1차접종자,10월말(누적) 2차접종자,10월말(누적) 추가접종,lat,lot,total
0,종로구,64067,28168,79113,40119,114242,67930,113892,74683,118945,112459,198,37.580695,126.982799,813816
1,중구,55134,18966,71425,27964,83716,48807,99187,63028,104060,97949,724,37.563656,126.997510,670960
2,용산구,65070,23205,82264,32113,122796,67814,173186,114336,180716,170709,184,37.554555,126.970779,1032393
3,성동구,85252,29483,107896,38956,162919,87625,224883,144948,233290,221269,1710,37.563500,127.036500,1338231
4,광진구,96935,32176,126826,42969,193076,102210,273100,169311,285027,268408,276,37.538400,127.082800,1590314


In [28]:
map_center = [37.55, 126.98]

m = folium.Map(location=map_center, zoom_start=11)

for i in range(len(df)):
    radius = (df['total'][i] / 10000) ** 0.8
    folium.CircleMarker(
        location=[df['lat'][i], df['lot'][i]],
        radius=radius,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        popup=f"{df['자치구'][i]} - 총 접종자 수: {df['total'][i]}명",
    ).add_to(m)
m